In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
data_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),'OP-Berichte_ASK_fixed.xlsx')

In [ ]:
os.getcwd()


In [ ]:
#Load all reports
xls = pd.ExcelFile(data_path)

In [ ]:
xls.sheet_names

In [ ]:
# Load the reports into a structured dataframe

df_final = pd.DataFrame(columns=['Indikation', 'Operation', 'ICD-Code', 'Diagnose', 'ICPM-Code', 'Prozedurentext'])
for x in xls.sheet_names:    
    df = xls.parse(sheet_name=x, header=None)
    df = df.transpose()
    headers = df.iloc[0].values
    df.columns = headers
    df.drop(index=0, axis=0, inplace=True)
    df_final.loc[len(df_final.index)] = df[df.columns[[0,1,3,4,5,6]]].values[0]

In [7]:
df = df_final

In [ ]:
df[0:16]

In [9]:
#Removes unwanted Characters from the sentences
#Removes \n
df['Indikation'] = df['Indikation'].str.replace(r'\n', '', regex=True)
df['Operation'] = df['Operation'].str.replace(r'\n', '', regex=True)


#Removes empty Rows 
df.replace('', np.nan, inplace=True)
#df.dropna(subset=['Indikation'], inplace=True)
df.dropna(subset=['Operation'], inplace=True)
'''df.dropna(subset=['ICD-Code'], inplace=True)
df.dropna(subset=['ICPM-Code'], inplace=True)
df.reset_index(drop = True, inplace=True)'''

#Removes Duplicates
df.drop_duplicates(inplace=True, ignore_index=True)

In [12]:
df.to_csv(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),'cleaned.csv'), index=False)

In [39]:
ops = df['Operation'].to_list()

In [ ]:
len(ops)

Compare the reports 

In [ ]:
import evaluate
from sklearn.model_selection import LeaveOneOut
import random

#Split reports into 2 random halves

random.shuffle(ops)

preds =  ops[:43]
refs = ops[43:86]

rouge = evaluate.load('rouge')

#Compute ROGUE Metrics
result = rouge.compute(predictions=list(preds), references=list(refs))

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

model = SentenceTransformer("all-MiniLM-L6-v2")

# Compute embeddings for both lists
embeddings1 = model.encode(refs)
embeddings2 = model.encode(preds)

# Compute cosine similarities
similarities = model.similarity(embeddings1, embeddings2)

torch.mean(similarities)